![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


In [1]:
SELECT * FROM companies

,company_id,company,city,country,continent
0,189,Otto Bock HealthCare,Duderstadt,Germany,Europe
1,848,Matrixport,,Singapore,Asia
2,556,Cloudinary,Santa Clara,United States,North America
3,999,PLACE,Bellingham,United States,North America
4,396,candy.com,New York,United States,North America
...,...,...,...,...,...
1069,1002,Poizon,Shanghai,China,Asia
1070,215,SSENSE,Montreal,Canada,North America
1071,866,Assembly,Culver City,United States,North America
1072,651,Xiaobing,Beijing,China,Asia


In [2]:
SELECT * FROM dates;

,company_id,date_joined,year_founded
0,189,2017-06-24 00:00:00+00:00,1919
1,848,2021-06-01 00:00:00+00:00,2019
2,556,2022-02-15 00:00:00+00:00,2011
3,999,2021-11-17 00:00:00+00:00,2020
4,396,2021-10-21 00:00:00+00:00,2021
...,...,...,...
1069,1002,2019-04-29 00:00:00+00:00,2015
1070,215,2021-06-08 00:00:00+00:00,2003
1071,866,2021-09-21 00:00:00+00:00,2019
1072,651,2021-07-12 00:00:00+00:00,2020


In [3]:
SELECT * FROM industries;

,company_id,industry
0,189,Health
1,848,Fintech
2,556,Internet software & services
3,999,Internet software & services
4,396,Fintech
...,...,...
1069,1002,Mobile & telecommunications
1070,215,E-commerce & direct-to-consumer
1071,866,Internet software & services
1072,651,Fintech


In [4]:
SELECT * FROM funding;

,company_id,valuation,funding,select_investors
0,189,4000000000,0,EQT Partners
1,848,1000000000,100000000,"Dragonfly Captial, Qiming Venture Partners, DS..."
2,556,2000000000,100000000,"Blackstone, Bessemer Venture Partners"
3,999,1000000000,100000000,"Goldman Sachs Asset Management, 3L"
4,396,2000000000,100000000,"Insight Partners, Softbank Group, Connect Vent..."
...,...,...,...,...
1069,1002,1000000000,0,"DST Global, Sequoia Capital China, Gaorong Cap..."
1070,215,4000000000,0,Sequoia Capital
1071,866,1000000000,0,"Advent International, PSG, Providence Equity P..."
1072,651,1000000000,0,"NetEase Capital, Northern Light Venture Capita..."


In [15]:
WITH top_industries AS (
	SELECT industry, COUNT(*) AS num_unicorns
	FROM dates AS d
	INNER JOIN industries AS i
	ON d.company_id = i.company_id
	WHERE EXTRACT(YEAR FROM CAST(date_joined AS DATE)) BETWEEN 2019 AND 2021
	GROUP BY industry
	ORDER BY num_unicorns DESC
	LIMIT 3
),
yearly_rankings AS (
	SELECT industry,
		   COUNT(*) AS num_unicorns, 
		   AVG(valuation) AS avg,
		   EXTRACT(YEAR FROM CAST(date_joined AS DATE)) AS year
	FROM industries AS i
	INNER JOIN funding AS f
	ON i.company_id = f.company_id
	INNER JOIN dates AS d
	ON i.company_id = d.company_id
	GROUP BY industry, year
)
SELECT industry, year, num_unicorns, ROUND((avg/1000000000),2) AS average_valuation_billions
FROM yearly_rankings
WHERE year BETWEEN 2019 AND 2021
	AND industry in (SELECT industry FROM top_industries)
GROUP BY industry, num_unicorns, year, avg
ORDER BY industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23


In [14]:
WITH top_industries AS
(
    SELECT i.industry, 
        COUNT(i.*)
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021')
    GROUP BY industry
    ORDER BY count DESC
    LIMIT 3
),

yearly_rankings AS 
(
    SELECT COUNT(i.*) AS num_unicorns,
        i.industry,
        EXTRACT(year FROM d.date_joined) AS year,
        AVG(f.valuation) AS average_valuation
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    INNER JOIN funding AS f
        ON d.company_id = f.company_id
    GROUP BY industry, year
)

SELECT industry,
    year,
    num_unicorns,
    ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
FROM yearly_rankings
WHERE year in ('2019', '2020', '2021')
    AND industry in (SELECT industry
                    FROM top_industries)
GROUP BY industry, num_unicorns, year, average_valuation
ORDER BY industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
